### Ave 1h forcing from ERA to 3h and 1d to compare

In [3]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cmocean
import glob
import tqdm

### Main

In [4]:
folder_in  = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/drowned_1h/'
folder_out = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/ERA5-forcing/drowned_3h/'

resample_atm(folder_in, folder_out, 1979, source='ERA5', freq='3h')

Resampling u10


100%|██████████| 12/12 [16:58<00:00, 84.88s/it]


Resampling v10


100%|██████████| 12/12 [10:51<00:00, 54.31s/it]


In [1]:
def resample_atm(folder_in, folder_out, year, source='ERA5', freq='1d'):

    if source=='ERA5':
        variables =  ['d2m', 'msdwlwrf', 'msdwswrf', 'msl', 'msr', 'mtpr', 't2m', 'u10', 'v10']
        for var in variables:
            print(f'Resampling {var}')
            for month in tqdm.tqdm(range(1,13)):
                var_file = xr.open_dataset(f'{folder_in}{var}_y{year}m{month:02}.nc')
                var_mean = var_file[var].resample(time=freq).mean()
                var_mean.to_netcdf(f'{folder_out}{var}_{freq}_y{year}m{month:02}.nc')
    else:
        raise Exception('Only set up for ERA5 files currently')

    return